In [0]:
! git clone --recursive https://github.com/shreyavarshini/Cancer-Detection.git

Cloning into 'Cancer-Detection'...
remote: Enumerating objects: 18689, done.
remote: Total 18689 (delta 0), reused 0 (delta 0), pack-reused 18689
Receiving objects: 100% (18689/18689), 318.63 MiB | 36.22 MiB/s, done.
Checking out files: 100% (18677/18677), done.


In [0]:
# imports and initializations
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [0]:
! cd '/content/Cancer-Detection/data/train/benign/'

In [0]:
# Arguments for the model
class Args:
  batch_size = 20
  epochs = 20
  no_cuda = False
  seed = 1
  log_interval = 10
  lr = 1e-3

args=Args()
args.cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")

In [0]:
# Dataset loader
def load_dataset( folder ):
    data_path = '/content/Cancer-Detection/data/' + folder
    transform = transforms.Compose([
    transforms.Resize(( 256, 256 ) ),
    transforms.ToTensor()
    ])
    dataset = datasets.ImageFolder( root=data_path, transform=transform )

    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=20,
        num_workers=3,
        shuffle=True
    )
    return loader

train_loader = load_dataset( 'train' )
test_loader = load_dataset( 'test' )

In [0]:
class VAE(nn.Module):
    def __init__(self, nc, ngf, ndf, latent_variable_size):
        super(VAE, self).__init__()

        self.nc = nc
        self.ngf = ngf
        self.ndf = ndf
        self.latent_variable_size = latent_variable_size

        # encoder
        self.e1 = nn.Conv2d(nc, ndf, 4, 2, 1)
        self.bn1 = nn.BatchNorm2d(ndf)

        self.e2 = nn.Conv2d(ndf, ndf*2, 4, 2, 1)
        self.bn2 = nn.BatchNorm2d(ndf*2)

        self.e3 = nn.Conv2d(ndf*2, ndf*4, 4, 2, 1)
        self.bn3 = nn.BatchNorm2d(ndf*4)

        self.e4 = nn.Conv2d(ndf*4, ndf*4, 4, 2, 1)
        self.bn4 = nn.BatchNorm2d(ndf*4)

        # changed 12 = 4
        self.e5 = nn.Conv2d(ndf*4, ndf*4, 12, 2, 1)
        self.bn5 = nn.BatchNorm2d(ndf*4)

        self.e6 = nn.Conv2d( ndf*8, ndf*8, 4, 2, 1)
        self.bn6 = nn.BatchNorm2d(ndf*8)

        #changed  8 = 4
        self.fc1 = nn.Linear(ndf*4*4*4, latent_variable_size)
        self.fc2 = nn.Linear(ndf*4*4*4, latent_variable_size)

        # decoder
        #changed 4 = 8
        self.d1 = nn.Linear(latent_variable_size, ngf*4*2*4*4)

        self.up1 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd1 = nn.ReplicationPad2d(1)
        #changed 4 = 8
        self.d2 = nn.Conv2d(ngf*4*2, ngf*4, 3, 1)
        self.bn6 = nn.BatchNorm2d(ngf*4, 1.e-3)

        self.up2 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd2 = nn.ReplicationPad2d(1)
        #changed 4 = 8, 2 = 4
        self.d3 = nn.Conv2d(ngf*4, ngf*2, 3, 1)
        self.bn7 = nn.BatchNorm2d(ngf*2, 1.e-3)

        self.up3 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd3 = nn.ReplicationPad2d(1)
        # changed 4 = 2, none = 2
        self.d4 = nn.Conv2d(ngf*2, ngf, 3, 1)
        self.bn8 = nn.BatchNorm2d(ngf, 1.e-3)
        # 5, 256, 32, 32

        self.up4 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd4 = nn.ReplicationPad2d(1)
        #changed ngf = ngf * 2
        self.d5 = nn.Conv2d(ngf, ngf, 3, 1)
        self.bn9 = nn.BatchNorm2d(ngf, 1.e-3)
        # 5, 256, 64, 64

        self.up6 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd6 = nn.ReplicationPad2d(1)
        self.d7 = nn.Conv2d(ngf, ngf, 3, 1)
        self.bn10 = nn.BatchNorm2d(ngf, 1.e-3)
        # 5, 256, 128, 128

        self.up5 = nn.UpsamplingNearest2d(scale_factor=2)
        self.pd5 = nn.ReplicationPad2d(1)
        self.d6 = nn.Conv2d(ngf, nc, 3, 1)

        self.leakyrelu = nn.LeakyReLU(0.2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.leakyrelu(self.bn1(self.e1(x)))
        h2 = self.leakyrelu(self.bn2(self.e2(h1)))
        h3 = self.leakyrelu(self.bn3(self.e3(h2)))
        h4 = self.leakyrelu(self.bn4(self.e4(h3)))
        h5 = self.leakyrelu(self.bn5(self.e5(h4)))
        #h6 = self.leakyrelu(self.bn6(self.e6(h5)))
        #changed 4 = 8
        h6 = h5.view(-1, self.ndf*4*4*4)

        return self.fc1(h6), self.fc2(h6)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if args.cuda:
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h1 = self.relu(self.d1(z))
        #changed 4 = 8
        h1 = h1.view(-1, self.ngf*4*2, 4, 4)
        h2 = self.leakyrelu(self.bn6(self.d2(self.pd1(self.up1(h1)))))
        h3 = self.leakyrelu(self.bn7(self.d3(self.pd2(self.up2(h2)))))
        h4 = self.leakyrelu(self.bn8(self.d4(self.pd3(self.up3(h3)))))
        h5 = self.leakyrelu(self.bn9(self.d5(self.pd4(self.up4(h4)))))
        h6 = self.leakyrelu(self.bn10(self.d7(self.pd6(self.up6(h5)))))
        #changed h6 = h5
        return self.sigmoid(self.d6(self.pd5(self.up5(h6))))

    def get_latent_var(self, x):
        mu, logvar = self.encode(x.view(-1, self.nc, self.ndf, self.ngf))
        z = self.reparametrize(mu, logvar)
        return z

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.nc, self.ndf, self.ngf))
        z = self.reparametrize(mu, logvar)
        res = self.decode(z)
        return res, mu, logvar


model = VAE(nc=3, ngf=256, ndf=256, latent_variable_size=500)

if args.cuda:
    model.cuda()

reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False
def loss_function(recon_x, x, mu, logvar):
    BCE = reconstruction_function(recon_x, x)

    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD

optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [0]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        data = data.to(device)
        # update the gradients to zero
        optimizer.zero_grad()

        # forward pass
        recon_batch, mu, logvar = model(data)
        #print(recon_batch.shape)

        # reconstruction loss + kl divergence loss
        loss = loss_function(recon_batch, data, mu, logvar)
        
         # backward pass
        loss.backward()
        train_loss += loss.item()

        # update the weights
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    # set the evaluation mode
    model.eval()

    # test loss for the data
    test_loss = 0

    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = Variable(data)
            data = data.to(device)

            # forward pass
            recon_batch, mu, logvar = model(data)

            # total loss
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

if __name__ == "__main__":
    for epoch in range(1, args.epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(1, 500).to(device)
            sample = Variable(sample, volatile=True)
            sample = model.decode(sample).cpu()
            img = sample.view( 256, 256, 3 ).data
            plt.figure()
            plt.imshow(img )
            plt.show()

tensor([[[[0.4667, 0.4157, 0.3843,  ..., 0.1137, 0.1059, 0.0941],
          [0.4745, 0.4235, 0.4000,  ..., 0.1686, 0.1333, 0.1059],
          [0.4902, 0.4392, 0.4196,  ..., 0.2039, 0.1569, 0.1176],
          ...,
          [0.4549, 0.4549, 0.4431,  ..., 0.0549, 0.0667, 0.0784],
          [0.4902, 0.4980, 0.5059,  ..., 0.0667, 0.0784, 0.0902],
          [0.5569, 0.5490, 0.5490,  ..., 0.1255, 0.1333, 0.1255]],

         [[0.4667, 0.4157, 0.3843,  ..., 0.1137, 0.1059, 0.0941],
          [0.4745, 0.4235, 0.4000,  ..., 0.1686, 0.1333, 0.1059],
          [0.4902, 0.4392, 0.4196,  ..., 0.2039, 0.1569, 0.1176],
          ...,
          [0.4549, 0.4549, 0.4431,  ..., 0.0549, 0.0667, 0.0784],
          [0.4902, 0.4980, 0.5059,  ..., 0.0667, 0.0784, 0.0902],
          [0.5569, 0.5490, 0.5490,  ..., 0.1255, 0.1333, 0.1255]],

         [[0.4667, 0.4157, 0.3843,  ..., 0.1137, 0.1059, 0.0941],
          [0.4745, 0.4235, 0.4000,  ..., 0.1686, 0.1333, 0.1059],
          [0.4902, 0.4392, 0.4196,  ..., 0

In [0]:
!ps -aux|grep python

root          18  0.4  0.7 402900 98776 ?        Sl   15:19   0:02 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         121  7.6 23.3 42509664 3111508 ?    Ssl  15:23   0:12 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-ea850a1b-b975-4d9c-a93f-20e5ff53e863.json
root         239  0.0  0.0  39196  6568 ?        S    15:26   0:00 /bin/bash -c ps -aux|grep python


In [0]:
!kill -9 121